In [ ]:
!pip install -U mistralai faiss_cpu

In [38]:
from mistralai import Mistral
api_key=getpass("Type Mistral API key...")
mistral = Mistral(api_key=api_key)

Type Mistral API key... ········


In [76]:
import re
def create_chunks():
    with open("documents.md", "r") as file:
        docs = []
        heading = topic = content = ""
        while True:
            line = file.readline()
            if line:
                if re.search(r"^# \w", line):
                    heading = re.sub(r"^#", "", line).strip()
                elif re.search(r"^## \w", line):
                    topic = re.sub("^##", "", line).strip()
                elif re.search(r"^\s*$", line) is None:
                    content = f"{content} {line}"
                else:
                    if content:
                        chunk = f"{heading}/{topic}\n{content}"
                        docs.append(chunk)
                    content = line
            else:
                break

    return docs

In [77]:
chunks  = create_chunks()

In [78]:
import numpy as np
import time


def get_embeddings(input):
    time.sleep(10)
    embeddings = mistral.embeddings.create(model="mistral-embed", inputs=input)
    return embeddings.data[0].embedding

doc_embeddings =  np.array([get_embeddings(chunk) for chunk in chunks])

In [79]:
import faiss

d = doc_embeddings.shape[1]
index = faiss.IndexFlatL2(d)
index.add(doc_embeddings)

In [88]:
def retrieve_context(question):
    embed = np.array([get_embeddings(question)])
    scores, indices = index.search(embed, k=2)
    similar_context = [chunks[i] for i in indices.tolist()[0]]
    return similar_context

In [81]:
results = retrieve_context("Give me some book titles and authors")

context  ['Nerdy Books/Book information\n     \n Title: To Kill a Mockingbird \n Author: Harper Lee \n Genre: Historical Fiction, Southern Gothic \n Publication Date: July 11, 1960 \n Summary: To Kill a Mockingbird is set in the 1930s in the fictional town of Maycomb, Alabama, and is narrated by Scout Finch, a young girl. The story explores themes of racial injustice, morality, and empathy, particularly through the trial of an African-American man, Tom Robinson, accused of raping a white woman. Scoutâ€™s father, Atticus Finch, defends Robinson, challenging the prejudices of the town. \n Customer Reviews:\n      - "A powerful story of justice and humanity that stays with you long after you finish." - Sarah J.\n      - "Timeless and profound, an absolute must-read." - Tom D. \n', 'Nerdy Books/Book information\n     \n Title: The Catcher in the Rye\n Author: J.D. Salinger \n Genre: Literary Fiction \n Publication Date: July 16, 1951 \n Summary: The Catcher in the Rye follows the story of 

In [82]:
results

['Nerdy Books/Book information\n     \n Title: To Kill a Mockingbird \n Author: Harper Lee \n Genre: Historical Fiction, Southern Gothic \n Publication Date: July 11, 1960 \n Summary: To Kill a Mockingbird is set in the 1930s in the fictional town of Maycomb, Alabama, and is narrated by Scout Finch, a young girl. The story explores themes of racial injustice, morality, and empathy, particularly through the trial of an African-American man, Tom Robinson, accused of raping a white woman. Scoutâ€™s father, Atticus Finch, defends Robinson, challenging the prejudices of the town. \n Customer Reviews:\n      - "A powerful story of justice and humanity that stays with you long after you finish." - Sarah J.\n      - "Timeless and profound, an absolute must-read." - Tom D. \n',
 'Nerdy Books/Book information\n     \n Title: The Catcher in the Rye\n Author: J.D. Salinger \n Genre: Literary Fiction \n Publication Date: July 16, 1951 \n Summary: The Catcher in the Rye follows the story of Holden C

In [89]:
Bookstore_Name = "Nerdy Book Store"

In [93]:
def get_chat_response(question):
    context = retrieve_context(question)

    sys_prompt = f"""
    ###
    Context Information: {context}
    Bookstore_Name: {Bookstore_Name}
    ###
    
    You are a helpful chat agent for {Bookstore_Name}. 
    Use the context information given above, answer the query related to Nerdy BookStore.
    User Query: {question}
    """

    messages = [{"role": "user", "content": sys_prompt}]
    time.sleep(10)
    chat_response = mistral.chat.complete(model="mistral-small-latest", messages=messages)
    return chat_response.choices[0].message.content

In [94]:
get_chat_response("What is the address of the Nerdy Book Store?")

'The address of Nerdy Book Store is:\n\nNerdy Book Store,\n123 Main Street,\nBook City, BC 12345'

In [95]:
get_chat_response("Do you have chat support?")

'Yes, we do have chat support! You can chat with us directly through our website at www.bookstore.com. Our chat support is available Monday to Friday, from 9:00 AM to 6:00 PM.'

In [96]:
get_chat_response("Do you have an events in December?")

"Yes, we do have an event in December! Here are the details:\n\n**Children's Storytime Hour**\n- Date: December 20, 2024\n- Time: 10:00 AM – 11:00 AM\n- Location: Nerdy Book Store, Kids’ Section\n- Storybooks: Where the Wild Things Are by Maurice Sendak, The Very Hungry Caterpillar by Eric Carle\n- Details: Join us for a fun-filled hour of reading and interactive storytelling for children! This event is perfect for kids ages 3-After the stories, we’ll have a craft activity related to the books we read. Parents are welcome to join the fun!\n- To Register: No registration required. Just drop in!"

In [97]:
get_chat_response("What books are available in the store?")

"I'm sorry for any confusion, but the context information provided does not include details about the books available in the store. For information on the books available at Nerdy Book Store, I would recommend visiting their website or contacting their customer support. They offer live chat support Monday to Friday from 9:00 AM to 6:00 PM, which you can access through their website at www.bookstore.com."

In [98]:
get_chat_response("Does the store have fictional books?")

'Yes, the Nerdy Book Store offers a variety of fictional books. For example, during the Children\'s Storytime Hour event, fictional storybooks like "Where the Wild Things Are" by Maurice Sendak and "The Very Hungry Caterpillar" by Eric Carle are read to the children. Additionally, the Monthly Book Club Meeting discusses fictional novels such as "The Night Circus" by Erin Morgenstern. These events showcase the store\'s commitment to providing a wide range of fictional literature for different age groups.'